# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

API key looks good so far


In [11]:
system_prompt = """You are a coding assistant. You will be provided with a code snippet and your task is to explain the code in detail.\
The code snippet will be in Python. You should provide a detailed explanation of the code. It should be in a markdown format. \
It might happen that the code is not correct. In that case, you should provide a detailed explanation of the code and then provide the correct code snippet.\
If not asked for more, anwer in up to 250 words."""

In [6]:
def user_prompt(code: str) -> str:
    return f"Explain the following code in detail:\n\n{code}\n\n###\n\n"

In [7]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [14]:
# Get gpt-4o-mini to answer, with streaming
def gpt_code_assist(code):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role": "user", "content": user_prompt(code)}
          ],
        stream=True,
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [13]:
# Get Llama 3.2 to answer
def llama_code_assist(code):
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role": "user", "content": user_prompt(code)}
          ],
        stream=True,
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [16]:
def code_assist(code, model = 'llama'):
    
    if model == 'gpt':
        openai = OpenAI()
        stream = openai.chat.completions.create(
            model=MODEL_GPT,
            messages=[
                {"role": "system", "content":system_prompt},
                {"role": "user", "content": user_prompt(code)}
            ],
            stream=True,
        )
    
    elif model == 'llama':
        
        ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
        stream = ollama_via_openai.chat.completions.create(
            model=MODEL_LLAMA,
            messages=[
                {"role": "system", "content":system_prompt},
                {"role": "user", "content": user_prompt(code)}
            ],
            stream=True,
        )
        
    else:
        raise ValueError("Model not available. Please use 'gpt' or 'llama'.")
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
code_assist(question, model='gpt')

The provided code snippet uses Python's `yield from` syntax along with a set comprehension. Let's break it down step by step:

### Components of the Code

1. **Set Comprehension**:
   python
   {book.get("author") for book in books if book.get("author")}
   
   - This creates a set of unique authors from a collection of `books`.
   - It iterates over each `book` in the `books` iterable.
   - The expression `book.get("author")` retrieves the value associated with the key `"author"` from each `book` dictionary.
   - The `if book.get("author")` condition ensures that only authors that are present (i.e., not `None`) are included in the set, preventing empty or missing authors from being considered.

2. **`yield from`**:
   python
   yield from ...
   
   - The `yield from` expression is typically used inside a generator function to yield values from an iterable one by one. 
   - In this context, it will generate each author in the set produced by the set comprehension.

### Purpose of the Code
The overall purpose of this code is to create a generator that yields each unique author's name from the provided `books` iterable, ensuring that only valid (non-None) author names are included.

### Code Correction
This code snippet is mostly correct but should ideally be wrapped in a generator function for `yield from` to work properly. Here’s how it might look:

python
def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}


### Summary
This code snippet is useful for extracting unique authors from a list of book dictionaries efficiently, enabling further operations or iterations over the authors in a generator style.

In [19]:
from code_assist import code_assist as ca

In [21]:
ca(question, model='gpt')

The provided code snippet uses a Python generator along with a set comprehension to yield authors from a collection of book dictionaries. Here's a detailed explanation of each component of the code:

python
yield from {book.get("author") for book in books if book.get("author")}


### Breakdown of the Code:

1. **Set Comprehension**:
   - `{book.get("author") for book in books if book.get("author")}`: This part of the code creates a set by iterating over a collection called `books`. It extracts the value of the `"author"` key for each `book` dictionary.
   - The condition `if book.get("author")` ensures that only authors that are not `None` or empty strings are included in the set. If a `book` does not contain a valid author, it will be excluded from the final set. Since a set only stores unique values, any duplicate authors will be stored only once.

2. **Yielding from a Set**:
   - `yield from`: This keyword allows a generator to yield multiple values from another iterable (like a set). In this case, the values being yielded are the unique authors extracted from the set comprehension.

### Purpose:
The overall purpose of this code is to create a generator that yields each unique, valid author from a list of books. This is useful in scenarios where one needs to process or display each author without duplicates, which can occur when there are multiple books by the same author in the `books` collection.

### Correctness:
The code appears correct syntactically and logically, assuming that `books` is a list or similar iterable containing dictionaries with an `"author"` key. If used within a generator function, it would work as intended.